In [6]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from torch import nn,optim
import torch

import algo

rand_seed=42
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
import numpy as np
import pandas as pd

import attack
import random




In [3]:
class Net_attack(nn.Module):

    def __init__(self, h_neurons, do, input_size):
        super(Net_attack, self).__init__()
        self.input_size = input_size
        self.h_neurons = h_neurons
        self.do = do
        self.fc1 = nn.Linear(input_size, h_neurons)
        self.fc2 = nn.Linear(h_neurons, 2)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(do)
        self.softmax = nn.Softmax(dim=1)   

    def forward(self, x):
        x = x.view(-1, self.input_size)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.drop(x)
        x = self.softmax(x)
        return x

class Train_args():

    def __init__(self, learning_rate, weight_decay, epoch):
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.epoch = epoch
         
def train_attack_model(model, train_data, train_target, train_args):

    optimizer = optim.Adam(model.parameters(), lr=train_args.learning_rate, weight_decay=train_args.weight_decay)
    
    model.train()
    optimizer.zero_grad()
    output = model(train_data)
    loss = nn.CrossEntropyLoss()(output, train_target.to(torch.long))
    loss.backward(retain_graph=True)
    optimizer.step()

    return model

def attack_evaluation(model, x, y, dev="cpu", extended=False):

    model.eval()

    with torch.no_grad():
        output =  model(x)
        out_target = output.argmax(1, keepdim=True)
        correct = out_target.to(dev).eq(y.to(dev).view_as(out_target.to(dev))).sum().item()
        acc = correct/y.shape[0]

        predicted_positive = output.argmax(1, keepdim=True) == 1
        labeled_positive = y == 1
        tp = predicted_positive.to(dev) * labeled_positive.to(dev).view_as(out_target)
        tp_count = tp.to(dev).sum().item()
        
        if predicted_positive.to(dev).sum().item() != 0:
            pre = tp_count / predicted_positive.to(dev).sum().item()
        else:
            pre = 0
        if labeled_positive.to(dev).sum().item() !=0:
            rec = tp_count / labeled_positive.to(dev).sum().item()
        else:
            rec = 0
    if extended:
        predicted_negative = output.argmax(1, keepdim=True) == 0
        labeled_negative = y == 0
        tn = predicted_negative.to(dev) * labeled_negative.to(dev).view_as(out_target)
        tn_count = tn.to(dev).sum().item()

        fp_count = predicted_positive.to(dev).sum().item() - tp.to(dev).sum().item()
        fn_count = labeled_positive.to(dev).sum().item() - tp.to(dev).sum().item()
        
        return acc, pre, rec, tp_count, tn_count, fp_count, fn_count
    else:
        return acc, pre, rec


In [7]:
# raw_data = pd.read_csv('../datasets/dataset_purchase', )
# y=raw_data['63']
# X=raw_data.drop('63', axis=1)
# y =  y.replace(100, 0)
# print(y.nunique())

# X_train, x_shadow, y_train, y_shadow = train_test_split(X, y, train_size=0.1, random_state=42)
# print(X_train.shape, x_shadow.shape)

100
(19732, 600) (177591, 600)


In [4]:
data = pd.read_csv('../DP-UTIL.nosync/loan_preprocessed.csv', index_col=0)


In [13]:
data.shape

(661283, 167)

In [16]:
#Loan
y = data['grade']
X = data.drop('grade', axis=1)
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X[0:100000], y[0:100000], train_size=0.5, random_state=0)
x_target_train = np.array(X_train)
y_target_train = np.array(y_train)
x_target_test = np.array(X_test)
y_target_test = np.array(y_test)

x_shadow = X[100000:]
y_shadow = y[100000:]

In [156]:
model = algo.LogisticRegression_DPSGD()

model.n_classes      = len(np.unique(y_target_test))
model.alpha          = 0.001
model.max_iter       = 100
model.lambda_        = 1e-5
model.tolerance      = 1e-5
model.DP             = False
model.L              = 1
model.epsilon        = 0

In [157]:
rand_seed=42
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
random.seed(rand_seed)

X,y = model.init_theta(x_target_train, y_target_train)
model.train(X,y)
model.evaluate(x_target_train, y_target_train, acc=True)
model.evaluate(x_target_test, y_target_test, acc=True)
            

The accuracy of the model : 88.6 %
The accuracy of the model : 88.1 %


0.88086

In [54]:
model.evaluate(x_target_train, y_target_train, acc=True)
model.evaluate(x_target_test, y_target_test, acc=True)

The accuracy of the model : 86.6 %
The accuracy of the model : 86.0 %


0.85972

In [142]:
s_ms = {}
number_of_sms = 20
shadow_size = 50000
shadow_batch_size = int(shadow_size/number_of_sms)

x_shadow_train = np.array(x_shadow[:shadow_size])
y_shadow_train = np.array(y_shadow[:shadow_size])
x_shadow_test = np.array(x_shadow[shadow_size:2*shadow_size])
y_shadow_test = np.array(y_shadow[shadow_size:2*shadow_size])

for i in range(number_of_sms):
    batch_start = i*shadow_batch_size
    batch_end = (i+1)*shadow_batch_size
    
    shadow_model = algo.LogisticRegression_DPSGD()

    shadow_model.n_classes      = len(np.unique(y_target_test))
    shadow_model.alpha          = 0.001
    shadow_model.max_iter       = 500
    shadow_model.lambda_        = 10e-5
    shadow_model.tolerance      = 10e-5
    shadow_model.DP             = False

    X,y = shadow_model.init_theta(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end] )
    shadow_model.SGD(X,y)
    print('Shadow model: ', i)
    shadow_model.evaluate(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end], acc=True)
    shadow_model.evaluate(x_shadow_test[batch_start:batch_end], y_shadow_test[batch_start:batch_end], acc=True)
    s_ms[i] = shadow_model

/var/folders/66/0p0lnj8d6zj532q0yvm1zt580000gn/T/ipykernel_30021/3790754660.py:7: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_shadow_train = np.array(y_shadow[:shadow_size])
/var/folders/66/0p0lnj8d6zj532q0yvm1zt580000gn/T/ipykernel_30021/3790754660.py:9: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_shadow_test = np.array(y_shadow[shadow_size:2*shadow_size])


Shadow model:  0
The accuracy of the model : 91.60000000000001 %
The accuracy of the model : 78.60000000000001 %
Shadow model:  1
The accuracy of the model : 91.8 %
The accuracy of the model : 79.2 %
Shadow model:  2
The accuracy of the model : 92.7 %
The accuracy of the model : 79.10000000000001 %
Shadow model:  3
The accuracy of the model : 92.4 %
The accuracy of the model : 80.10000000000001 %
Shadow model:  4
The accuracy of the model : 91.10000000000001 %
The accuracy of the model : 79.4 %
Shadow model:  5
The accuracy of the model : 91.8 %
The accuracy of the model : 79.2 %
Shadow model:  6
The accuracy of the model : 92.2 %
The accuracy of the model : 78.7 %
Shadow model:  7
The accuracy of the model : 91.60000000000001 %
The accuracy of the model : 79.3 %
Shadow model:  8
The accuracy of the model : 92.30000000000001 %
The accuracy of the model : 78.4 %
Shadow model:  9
The accuracy of the model : 92.2 %
The accuracy of the model : 81.2 %
Shadow model:  10
The accuracy of the m

In [143]:
shadow_train_pred = []
shadow_test_pred = []

for i in range(number_of_sms): 
    batch_start = i*shadow_batch_size
    batch_end = (i+1)*shadow_batch_size
    
    train_prediciton = s_ms[i].predict(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end])
    test_prediciton = s_ms[i].predict(x_shadow_test[batch_start:batch_end], y_shadow_test[batch_start:batch_end])
    
    shadow_train_pred.append(train_prediciton)
    shadow_test_pred.append(test_prediciton)
    
    

In [144]:
y_shadow_train_ohe = OneHotEncoder(sparse=False).fit_transform(y_shadow_train.reshape(-1,1)) #encoode the target values
y_shadow_test_ohe = OneHotEncoder(sparse=False).fit_transform(y_shadow_test.reshape(-1,1)) #encoode the target values


In [145]:
shadow_train_pred[0].shape

(2500, 7)

In [146]:
sh_train_pred = np.concatenate(shadow_train_pred)
sh_test_pred = np.concatenate(shadow_test_pred)

# members
labels = np.ones(sh_train_pred.shape[0])
# non-members
test_labels = np.zeros(sh_test_pred.shape[0])

x_1 = np.concatenate((sh_train_pred, sh_test_pred))
x_2 = np.concatenate((y_shadow_train_ohe, y_shadow_test_ohe))#.reshape((-1, 1))
y_new = np.concatenate((labels, test_labels))


In [147]:
attack_train_data = np.concatenate((x_1,x_2),axis=1)
attack_train_target = y_new
df = pd.DataFrame(attack_train_data)
df['a_target'] = attack_train_target
df = df.sample(frac = 1)

attack_train_data = torch.tensor(np.array(df.drop(['a_target'], axis=1)), dtype=torch.float, requires_grad=True)
attack_train_target = torch.tensor(np.array(df['a_target']), dtype=torch.float)

In [152]:
# attack_train_data = torch.tensor(np.array(df.drop(['a_target'], axis=1)), dtype=torch.float, requires_grad=True)   
# attack_train_target = torch.tensor(np.array(df['a_target']), dtype=torch.float)
attack_train_args = Train_args(learning_rate=0.001, weight_decay=1e-5, epoch=500)
attack_model = Net_attack(h_neurons=64, do=0, input_size=attack_train_data.shape[1])
for epoch in range(attack_train_args.epoch):
    attack_model = attack.train_attack_model(attack_model, attack_train_data, attack_train_target, attack_train_args)
    print(f'epoch {epoch}: {attack_evaluation(attack_model, attack_train_data, attack_train_target)}' )        


epoch 0: (0.49126, 0.48604368932038833, 0.30438)
epoch 1: (0.49315, 0.49249676868140296, 0.44962)
epoch 2: (0.49551, 0.49579154559940014, 0.52896)
epoch 3: (0.49855, 0.4987650535711244, 0.58562)
epoch 4: (0.50248, 0.5019040014740657, 0.65374)
epoch 5: (0.50778, 0.5055169479506453, 0.71288)
epoch 6: (0.51181, 0.5080920340399874, 0.74154)
epoch 7: (0.50895, 0.5079857950996662, 0.56932)
epoch 8: (0.51039, 0.5092709913446953, 0.57074)
epoch 9: (0.51307, 0.511784754657097, 0.5676)
epoch 10: (0.51587, 0.514453288646837, 0.56488)
epoch 11: (0.51749, 0.5161355794601177, 0.55946)
epoch 12: (0.52027, 0.5186679191762907, 0.56318)
epoch 13: (0.5219, 0.5205394657863145, 0.55502)
epoch 14: (0.52263, 0.5216526015653405, 0.5452)
epoch 15: (0.52283, 0.5223363663046668, 0.53388)
epoch 16: (0.52494, 0.5210563641889838, 0.61716)
epoch 17: (0.52525, 0.5213819967821154, 0.6157)
epoch 18: (0.52589, 0.5217947638690126, 0.61984)
epoch 19: (0.53099, 0.5257473288911783, 0.6328)
epoch 20: (0.5314, 0.5265131045663

epoch 168: (0.56995, 0.5504842737337433, 0.76274)
epoch 169: (0.56873, 0.5509979965867775, 0.74258)
epoch 170: (0.56977, 0.5503282117867705, 0.76292)
epoch 171: (0.56871, 0.5509544220814856, 0.74294)
epoch 172: (0.5698, 0.5502114925330187, 0.76486)
epoch 173: (0.56864, 0.5509183703747663, 0.74266)
epoch 174: (0.56995, 0.5503070926168319, 0.76518)
epoch 175: (0.56865, 0.5508737087044805, 0.74336)
epoch 176: (0.56997, 0.5503041108890391, 0.76544)
epoch 177: (0.56861, 0.550797387943672, 0.74394)
epoch 178: (0.56995, 0.5502868398728991, 0.76546)
epoch 179: (0.56867, 0.5507966801295993, 0.7446)
epoch 180: (0.57009, 0.5503397158739963, 0.76626)
epoch 181: (0.56884, 0.5509036055488183, 0.74502)
epoch 182: (0.57001, 0.5502865926362213, 0.76612)
epoch 183: (0.56925, 0.5510972064400926, 0.74688)
epoch 184: (0.56998, 0.5502210357101849, 0.7667)
epoch 185: (0.56915, 0.5510204081632653, 0.74682)
epoch 186: (0.57003, 0.5502057554163142, 0.76746)
epoch 187: (0.56912, 0.5509975209538425, 0.7468)
epoch

epoch 334: (0.57336, 0.5503583294434224, 0.80174)
epoch 335: (0.57218, 0.5508955013397264, 0.78128)
epoch 336: (0.57342, 0.5503732367308853, 0.80218)
epoch 337: (0.57222, 0.5508792199740743, 0.78194)
epoch 338: (0.57343, 0.5503462461432979, 0.80268)
epoch 339: (0.57228, 0.5509014084507042, 0.78228)
epoch 340: (0.57347, 0.5503529573024467, 0.80302)
epoch 341: (0.57227, 0.5508735868448099, 0.78256)
epoch 342: (0.57345, 0.5503240747084698, 0.80322)
epoch 343: (0.5723, 0.5508725021108922, 0.7829)
epoch 344: (0.57342, 0.550282160859084, 0.8035)
epoch 345: (0.5723, 0.5508324427695596, 0.78346)
epoch 346: (0.57341, 0.5502498459853515, 0.80386)
epoch 347: (0.57242, 0.5509067903838043, 0.78372)
epoch 348: (0.57344, 0.5502669404517454, 0.80394)
epoch 349: (0.57247, 0.5509111601309485, 0.7842)
epoch 350: (0.57341, 0.5502237182381675, 0.80424)
epoch 351: (0.57244, 0.5508893697136594, 0.78418)
epoch 352: (0.57338, 0.550201132911912, 0.80424)
epoch 353: (0.57262, 0.5509728500435186, 0.78496)
epoch 3

In [153]:
train_res = attack_evaluation(attack_model, attack_train_data, attack_train_target)

In [154]:
test_res = attack.mi_attack_test(model, attack_model, x_target_train, y_target_train, x_target_test, y_target_test)

In [155]:
print(train_res)
test_res


(0.57401, 0.551659151508383, 0.79034)


(0.50259, 0.5017606351882317, 0.73812)

In [61]:
sh_path = f'mia/shms{number_of_sms}_shtrsize{shadow_size}_shlr{shadow_model.alpha}_shiter{int(shadow_model.max_iter/shadow_batch_size)}_shreg{shadow_model.lambda_}/'      
os.mkdir(sh_path)

torch.save(attack_train_data, sh_path+'attack_train_data.pt')
torch.save(attack_train_target, sh_path+'attack_train_target.pt')

at_path = sh_path+f'attack_model_aneur{attack_model.h_neurons}_ado{attack_model.do}_alr{attack_train_args.learning_rate}_alreg{attack_train_args.weight_decay}_aepoch{attack_train_args.epoch}'
torch.save(attack_model, at_path)


In [74]:
train_acc, train_pre, train_rec = attack_evaluation(attack_model, attack_train_data, attack_train_target)
train_acc, train_pre, train_rec 

(0.59217, 0.5930014328093153, 0.5877)

In [75]:
test_acc, test_pre, test_rec = attack_evaluation(attack_model, attack_test_data, attack_test_target)
test_acc, test_pre, test_rec

(0.6260389215487533, 0.5935737236701178, 0.8313)

In [ ]:
# MI optimization

In [ ]:
ams

In [87]:
rand_seed=42
np.random.seed(rand_seed)
# random.seed(rand_seed)
torch.manual_seed(rand_seed)

epo = [200]
wd = [1e-5, 1e-6]
ams = {}
for ep in epo:
    for w_d in wd:
        attack_train_args = Train_args(learning_rate=l_r, weight_decay=w_d, epoch=ep)
        attack_model = Net_attack(h_neurons=64, do=0, input_size=attack_train_data.shape[1])
        for epoch in range(attack_train_args.epoch):

            attack_model = train_model(attack_model, attack_train_data, attack_train_target, attack_train_args)
            ams[(l_r,w_d)] = attack_model
        at_path = sh_path+f'attack_model_aneur{attack_model.h_neurons}_ado{attack_model.do}_alr{attack_train_args.learning_rate}_alreg{attack_train_args.weight_decay}_aepoch{attack_train_args.epoch}'
        torch.save(attack_model, at_path)


In [91]:
epo = [200,500]
wd = [1e-5, 1e-6]

for ep in epo:
    for w_d in wd:
        print(ep, w_d)
        attack_train_args = Train_args(learning_rate=l_r, weight_decay=w_d, epoch=500)
        attack_model = Net_attack(h_neurons=64, do=0, input_size=attack_train_data.shape[1])
        at_path = sh_path+f'attack_model_aneur{attack_model.h_neurons}_ado{attack_model.do}_alr{attack_train_args.learning_rate}_alreg{attack_train_args.weight_decay}_aepoch{attack_train_args.epoch}'
        attack_model = torch.load(at_path)
        
        train_acc, train_pre, train_rec = attack_evaluation(attack_model, attack_train_data, attack_train_target)
        print('Train: ', train_acc, train_pre, train_rec)
        test_acc, test_pre, test_rec = attack_evaluation(attack_model, attack_test_data, attack_test_target)
        print('Test: ' , test_acc, test_pre, test_rec)


200 1e-05
Train:  0.8081 0.8207236842105263 0.78842
Test:  0.6895398337725522 0.6209717711716213 0.9943
200 1e-06
Train:  0.80328 0.8273606493674712 0.7665
Test:  0.7008919521589296 0.6304929308368361 0.99
500 1e-05
Train:  0.8081 0.8207236842105263 0.78842
Test:  0.6895398337725522 0.6209717711716213 0.9943
500 1e-06
Train:  0.80328 0.8273606493674712 0.7665
Test:  0.7008919521589296 0.6304929308368361 0.99


In [ ]:
for l_r in lr:
    for w_d in wd:
        ams[(l_r,w_d)]
        train_acc, train_pre, train_rec = attack_evaluation(attack_model, attack_train_data, attack_train_target)
        print("Train acc pre rec: ", train_acc, train_pre, train_rec)
        test_acc, test_pre, test_rec = attack_evaluation(attack_model, attack_test_data, attack_test_target)
        print("Test acc pre rec: ", test_acc, test_pre, test_rec)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
shadow_size = 20000
shadow_clf = LogisticRegression(random_state=1).fit(x_shadow[:shadow_size], y_shadow[:shadow_size])

In [ ]:
print(shadow_clf.score(x_shadow[:shadow_size], y_shadow[:shadow_size]))
print(shadow_clf.score(x_shadow[shadow_size:shadow_size*2], y_shadow[shadow_size:shadow_size*2]))

In [ ]:
shadow_train_pred = shadow_clf.predict_proba(x_shadow[:shadow_size])
shadow_test_pred = shadow_clf.predict_proba(x_shadow[shadow_size:shadow_size*2])
y_shadow_train = y_shadow[:shadow_size]
y_shadow_test = y_shadow[shadow_size:shadow_size*2]

In [ ]:
# members
labels = np.ones(shadow_train_pred.shape[0])
# non-members
test_labels = np.zeros(shadow_test_pred.shape[0])

x_1 = np.concatenate((shadow_train_pred, shadow_test_pred))
x_2 = np.concatenate((y_shadow_train, y_shadow_test)).reshape((-1, 1))
y_new = np.concatenate((labels, test_labels))

attack_train_data = np.concatenate((x_1,x_2),axis=1)
attack_train_target = y_new
df = pd.DataFrame(attack_train_data)
df['a_target'] = attack_train_target
df = df.sample(frac = 1)

attack_train_data = np.array(df.drop(['a_target'], axis=1))
attack_train_target = np.array(df['a_target'])


In [ ]:
attack_train_data = torch.tensor(np.array(df.drop(['a_target'], axis=1)), dtype=torch.float, requires_grad=True)   
attack_train_target = torch.tensor(np.array(df['a_target']), dtype=torch.float)
attack_train_args = Train_args(learning_rate=0.001, weight_decay=1e-3, epoch=200)
attack_model = Net_attack(h_neurons=64, do=0, input_size=attack_train_data.shape[1])


In [ ]:
attack_train_data = torch.tensor(np.array(df.drop(['a_target'], axis=1)), dtype=torch.float, requires_grad=True)   
attack_train_target = torch.tensor(np.array(df['a_target']), dtype=torch.float)
attack_train_args = Train_args(learning_rate=0.001, weight_decay=1e-3, epoch=500)
attack_model = Net_attack(h_neurons=64, do=0, input_size=attack_train_data.shape[1])
for epoch in range(attack_train_args.epoch):
            
    attack_model = train_model(attack_model, attack_train_data, attack_train_target, attack_train_args)
    train_acc, train_pre, train_rec = attack_evaluation(attack_model, attack_train_data, attack_train_target)

In [ ]:
train_acc, train_pre, train_rec 

In [67]:
train_pred = model.predict(x_target_train, y_target_train)
test_pred = model.predict(x_target_test, y_target_test)

y_target_train_ohe = OneHotEncoder(sparse=False).fit_transform(y_target_train.reshape(-1,1)) #encoode the target values
y_target_test_ohe = OneHotEncoder(sparse=False).fit_transform(y_target_test.reshape(-1,1)) #encoode the target values
    
# members
labels = np.ones(train_pred.shape[0])
# non-members
test_labels = np.zeros(test_pred.shape[0])

x_1 = np.concatenate((train_pred, test_pred))
x_2 = np.concatenate((y_target_train_ohe, y_target_test_ohe))#.reshape((-1, 1))
y_new = np.concatenate((labels, test_labels))

attack_test_data = np.concatenate((x_1,x_2),axis=1)
attack_test_target = y_new
df = pd.DataFrame(attack_test_data)
df['a_target'] = attack_test_target
df = df.sample(frac = 1)

attack_test_data = torch.tensor(np.array(df.drop(['a_target'], axis=1)), dtype=torch.float, requires_grad=True)
attack_test_target = torch.tensor(np.array(df['a_target']))


In [ ]:
test_acc, test_pre, test_rec = attack_evaluation(a_model, attack_test_data, attack_test_target)
test_acc, test_pre, test_rec

In [14]:
a_model = Net_attack(h_neurons=64, do=0, input_size=100)
a_model = torch.load('attack_model') 

In [ ]:

rand_seed=42
np.random.seed(rand_seed)
# random.seed(rand_seed)
torch.manual_seed(rand_seed)

lr = [0.001]
wd = [1e-4, 1e-6]
tms = {}
for l_r in lr:
    for w_d in wd:
        model = algo.LogisticRegression_DPSGD()

        model.n_classes      = 100
        model.alpha          = 0.001
        model.max_iter       = 100*X_train_size 
        model.lambda_        = 1e-3
        model.tolerance      = 10e-5
        model.DP             = True
        model.epsilon

        X,y = model.init_theta(x_target_train, y_target_train)
        model.SGD(X,y)
        model.evaluate(x_target_train, y_target_train)

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

import algo
# import attack

# from torch import nn,optim
import torch

rand_seed=42
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
        

raw_data_path = '../datasets/dataset_purchase'
raw_data = pd.read_csv(raw_data_path)
y=raw_data['63']
X=raw_data.drop('63', axis=1)
y =  y.replace(100, 0)
print('Dataset: ', raw_data_path)
print('Classes in classification task: ', y.nunique())
n_classes = y.nunique()

X_train, x_shadow, y_train, y_shadow = train_test_split(X, y, train_size=0.2, random_state=rand_seed)
print(X_train.shape, x_shadow.shape)

#Target model
X_train_size = 10000
X_test_size = 10000
x_target_train = np.array(X_train[:X_train_size])
y_target_train = np.array(y_train[:X_train_size])
x_target_test = np.array(X_train[X_train_size:X_train_size+X_test_size])
y_target_test = np.array(y_train[X_train_size:X_train_size+X_test_size])
if y_target_test.shape[0]<X_test_size or y_target_train.shape[0]<X_train_size:
    raise ValueError(
            "Not enough traning or test data for the target model")

for L in [1,10,100]:
    for epsilon in np.arange(0,1,0.1):
        model = algo.LogisticRegression_DPSGD()

        model.n_classes      = n_classes
        model.alpha          = 0.001
        model.max_iter       = 100*X_train_size
        model.lambda_        = 1e-3
        model.tolerance      = 10e-5
        model.DP             = True
        model.L              = L
        model.epsilon        = epsilon


        X,y = model.init_theta(x_target_train, y_target_train)
        model.train(X,y)
        model.evaluate(x_target_train, y_target_train, acc=True)
        model.evaluate(x_target_test, y_target_test, acc=True)

        tm_path = f'tm/lr{model.alpha}_iter{int(model.max_iter/X_train_size)}_reg{model.lambda_}_DP{model.DP}'
        if model.DP:
            tm_path += f'_eps{model.epsilon}_L{model.L}'
        np.save(tm_path+'_target_model', model.theta)

        print(tm_path)

    

In [ ]:
#Shadow models
# s_ms = {}
# number_of_sms = 10
# shadow_size = 50000
# shadow_batch_size = int(shadow_size/number_of_sms)

# x_shadow_train = np.array(x_shadow[:shadow_size])
# y_shadow_train = np.array(y_shadow[:shadow_size])
# x_shadow_test = np.array(x_shadow[shadow_size:2*shadow_size])
# y_shadow_test = np.array(y_shadow[shadow_size:2*shadow_size])

# attack.train_shadow_models(number_of_sms,)

# for i in range(number_of_sms):  
#     batch_start = i*shadow_batch_size
#     batch_end = (i+1)*shadow_batch_size
    
#     shadow_model = algo.LogisticRegression_DPSGD()

#     shadow_model.n_classes      = n_classes
#     shadow_model.alpha          = 0.001
#     shadow_model.max_iter       = 100*shadow_batch_size
#     shadow_model.lambda_        = 10e-3
#     shadow_model.tolerance      = 10e-5
#     shadow_model.DP             = False

#     X,y = shadow_model.init_theta(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end] )
#     shadow_model.SGD(X,y)
#     print('Shadow model: ', i)
#     shadow_model.evaluate(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end])
#     shadow_model.evaluate(x_shadow_test[batch_start:batch_end], y_shadow_test[batch_start:batch_end])
#     s_ms[i] = shadow_model

# #Attack model

# shadow_train_pred = []
# shadow_test_pred = []

# for i in range(number_of_sms): 
#     batch_start = i*shadow_batch_size
#     batch_end = (i+1)*shadow_batch_size
    
#     train_prediciton = s_ms[i].predict(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end])
#     test_prediciton = s_ms[i].predict(x_shadow_test[batch_start:batch_end], y_shadow_test[batch_start:batch_end])
    
#     shadow_train_pred.append(train_prediciton)
#     shadow_test_pred.append(test_prediciton)
    

In [10]:
1/10000 == 1e-5


False

In [3]:
np.arange(0,1,0.1)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])